# import necessary library

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Function to load images from a given folder
def load_images_from_folder(folder, label, img_size=(256, 256)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, img_size)
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

# Load cat and dog images
cats, cat_labels = load_images_from_folder('dataset/train/cat', label=0)
dogs, dog_labels = load_images_from_folder('dataset/train/dog', label=1)


In [3]:
# Combine the datasets
X = np.concatenate((cats, dogs), axis=0)
y = np.concatenate((cat_labels, dog_labels), axis=0)

In [4]:

# Flatten the images for SVM input
X_flattened = X.reshape(X.shape[0], -1)

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_flattened, y, test_size=0.2, random_state=42)

In [6]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
# Train the model
from sklearn.svm import SVC

# Train the SVM model with a linear kernel
svm = SVC(kernel='rbf', C=1.0, verbose=True, random_state=42)
svm.fit(X_train, y_train)

[LibSVM]..*
optimization finished, #iter = 2959
obj = -1976.760898, rho = 0.266156
nSV = 2965, nBSV = 2153
Total nSV = 2965


SVC(random_state=42, verbose=True)

In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Make predictions
y_pred = svm.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:\n', cm)

# Classification report
print('Classification Report:\n', classification_report(y_test, y_pred))


Accuracy: 0.63
Confusion Matrix:
 [[233 189]
 [106 273]]
Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.55      0.61       422
           1       0.59      0.72      0.65       379

    accuracy                           0.63       801
   macro avg       0.64      0.64      0.63       801
weighted avg       0.64      0.63      0.63       801



In [9]:
# improve the  model
svm = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm.fit(X_train, y_train)

SVC(random_state=42)

In [10]:
import joblib

# Save the trained model to a file
model_filename = 'svm_cats_dogs_model.pkl'
joblib.dump(svm, model_filename)

print(f"Model saved to {model_filename}")

Model saved to svm_cats_dogs_model.pkl


In [11]:
# Load the model from the file
loaded_model = joblib.load(model_filename)

print("Model loaded successfully")

Model loaded successfully


In [12]:
# Example test on new data (you can load new images or use the test set)
y_pred = loaded_model.predict(X_test)

# Evaluate the model's performance on the test set
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy on test set: {accuracy:.2f}')

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:\n', cm)

# Classification report
print('Classification Report:\n', classification_report(y_test, y_pred))

Accuracy on test set: 0.63
Confusion Matrix:
 [[233 189]
 [106 273]]
Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.55      0.61       422
           1       0.59      0.72      0.65       379

    accuracy                           0.63       801
   macro avg       0.64      0.64      0.63       801
weighted avg       0.64      0.63      0.63       801



In [19]:
def preprocess_image(image_path, img_size=(256, 256)):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, img_size)
    img = img.flatten()  # Flatten the image to match the model's input shape
    img = scaler.transform([img])  # Apply the same scaler used in training
    return img

# Load and preprocess a new image
new_image_path = 'dataset/test1/2010.jpg'  # Replace with the path to the new image
new_image = preprocess_image(new_image_path)

# Predict the class
prediction = loaded_model.predict(new_image)
print(f'Predicted class: {"Cat" if prediction[0] == 0 else "Dog"}')

Predicted class: Dog


In [20]:
pip install gradio

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 82.6 kB/s eta 0:00:00m eta 0:00:010:00:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 KB 23.6 kB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 KB 57.9 kB/s eta 0:00:000:00:010:00:01:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 KB 27.8 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 KB 94.2 kB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 22.8 kB/s eta 0:00:001m683.3 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 81.5 kB/s eta 0:00:00m eta 0:00:0136m0:00:04
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 KB 161.4 kB/s eta 0:00:0031m1.8 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 KB 73.3 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━

In [6]:
import gradio as gr
import cv2
import numpy as np

def classify_image(image):
    # Resize the image to 256x256
    img_resized = cv2.resize(image, (256, 256))
    
    # Convert the image to grayscale if necessary
    img_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
    
    # Flatten the image
    img_flattened = img_gray.flatten().reshape(1, -1)
    
    # Apply the same standardization used during training
    img_scaled = scaler.transform(img_flattened)
    
    # Make a prediction using the trained SVM model
    prediction = svm.predict(img_scaled)
    
    return "Cat" if prediction == 0 else "Dog"

interface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(),  # Define the input as an image
    outputs="text",  # Define the output as text
    title="Cat vs Dog Classifier",
)

interface.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
